In [1]:
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 8.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import re
import torch
from transformers import AutoTokenizer, AutoModel
import contractions
import numpy as np


In [3]:
df= pd.read_csv("/content/drive/MyDrive/EMR/patient_coding.csv")

In [4]:
df

,MRN,SOURCE_KEY,SOURCE_NAME,NAME,REF_BILL_CODE_SET_NAME,REF_BILL_CODE
0,798a98e2a1bcbac7,3,Final Diagnosis Primary Code Set,Infection and inflammatory reaction due to int...,ICD-10-CM,T84.621A
1,798a98e2a1bcbac7,3,Final Diagnosis Primary Code Set,"Postprocedural septic shock, initial encounter...",ICD-10-CM,T81.12XA
2,798a98e2a1bcbac7,3,Final Diagnosis Primary Code Set,Hypo-osmolality and hyponatremia,ICD-10-CM,E87.1
3,798a98e2a1bcbac7,3,Final Diagnosis Primary Code Set,"Nicotine dependence, cigarettes, uncomplicated",ICD-10-CM,F17.210
4,798a98e2a1bcbac7,11,ICD Procedure Primary Code Set,"Excision of Left Upper Femur, Open Approach",ICD-10-PCS,0QB70ZZ
...,...,...,...,...,...,...
2033943,cb1c94fc396c9230,3,Final Diagnosis Primary Code Set,Hyperkalemia,ICD-10-CM,E87.5
2033944,cb1c94fc396c9230,3,Final Diagnosis Primary Code Set,"Displaced fracture of shaft of right clavicle,...",ICD-10-CM,S42.021A
2033945,cb1c94fc396c9230,3,Final Diagnosis Primary Code Set,Patient's noncompliance with other medical tre...,ICD-10-CM,Z91.19
2033946,cb1c94fc396c9230,11,ICD Procedure Primary Code Set,"Drainage of R Pleural Cav with Drain Dev, Perc...",ICD-10-PCS,0W9940Z


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2033948 entries, 0 to 2033947
Data columns (total 6 columns):
 #   Column                  Dtype 
---  ------                  ----- 
 0   MRN                     object
 1   SOURCE_KEY              int64 
 2   SOURCE_NAME             object
 3   NAME                    object
 4   REF_BILL_CODE_SET_NAME  object
 5   REF_BILL_CODE           object
dtypes: int64(1), object(5)
memory usage: 93.1+ MB


In [6]:
df.describe()

,SOURCE_KEY
count,2.033948e+06
mean,9.911754e+00
std,7.797104e+00
min,3.000000e+00
25%,3.000000e+00
50%,5.000000e+00
75%,2.100000e+01
max,2.300000e+01


In [16]:
df[df['SOURCE_NAME']=='Final Diagnosis Primary Code Set']

,MRN,SOURCE_KEY,SOURCE_NAME,NAME,REF_BILL_CODE_SET_NAME,REF_BILL_CODE
0,798a98e2a1bcbac7,3,Final Diagnosis Primary Code Set,Infection and inflammatory reaction due to int...,ICD-10-CM,T84.621A
1,798a98e2a1bcbac7,3,Final Diagnosis Primary Code Set,"Postprocedural septic shock, initial encounter...",ICD-10-CM,T81.12XA
2,798a98e2a1bcbac7,3,Final Diagnosis Primary Code Set,Hypo-osmolality and hyponatremia,ICD-10-CM,E87.1
3,798a98e2a1bcbac7,3,Final Diagnosis Primary Code Set,"Nicotine dependence, cigarettes, uncomplicated",ICD-10-CM,F17.210
7,798a98e2a1bcbac7,3,Final Diagnosis Primary Code Set,Infection and inflammatory reaction due to int...,ICD-10-CM,T84.621A
...,...,...,...,...,...,...
2033942,cb1c94fc396c9230,3,Final Diagnosis Primary Code Set,"Other stimulant abuse, uncomplicated (CMS-HCC)",ICD-10-CM,F15.10
2033943,cb1c94fc396c9230,3,Final Diagnosis Primary Code Set,Hyperkalemia,ICD-10-CM,E87.5
2033944,cb1c94fc396c9230,3,Final Diagnosis Primary Code Set,"Displaced fracture of shaft of right clavicle,...",ICD-10-CM,S42.021A
2033945,cb1c94fc396c9230,3,Final Diagnosis Primary Code Set,Patient's noncompliance with other medical tre...,ICD-10-CM,Z91.19


In [7]:
df['SOURCE_KEY'].unique()

array([ 3, 11, 21, 23,  5, 13, 22])

In [30]:
df['REF_BILL_CODE_SET_NAME'].unique()
# df['REF_BILL_CODE_SET_NAME_encoded'].unique()
# df['REF_BILL_CODE_SET_NAME'].value_counts()


array([0, 1, 2])

In [18]:
df[df['SOURCE_NAME'] == 'Final Di']

,MRN,SOURCE_KEY,SOURCE_NAME,NAME,REF_BILL_CODE_SET_NAME,REF_BILL_CODE
511919,7ace7e3e41b4f9c0,3,Final Di,NaN,NaN,NaN


In [19]:
df = df[df['SOURCE_NAME'] != 'Final Di']

In [28]:
null_counts = df.isnull().sum()

# Display the result
print(null_counts)

MRN                               0
SOURCE_KEY                        0
SOURCE_NAME                       0
NAME                              0
REF_BILL_CODE_SET_NAME            0
REF_BILL_CODE                     0
REF_BILL_CODE_SET_NAME_encoded    0
dtype: int64


In [27]:
df['NAME'] = df['NAME'].fillna('missing')

<ipython-input-27-70fe3f9e2566>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['NAME'] = df['NAME'].fillna('missing')


In [25]:
df['REF_BILL_CODE_SET_NAME'] = df['REF_BILL_CODE_SET_NAME'].fillna('missing')

label_encoder_code_set_name = LabelEncoder()

df['REF_BILL_CODE_SET_NAME_encoded'] = label_encoder_code_set_name.fit_transform(df['REF_BILL_CODE_SET_NAME'])

<ipython-input-25-063165feeca4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['REF_BILL_CODE_SET_NAME'] = df['REF_BILL_CODE_SET_NAME'].fillna('missing')
<ipython-input-25-063165feeca4>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['REF_BILL_CODE_SET_NAME_encoded'] = label_encoder_code_set_name.fit_transform(df['REF_BILL_CODE_SET_NAME'])


In [35]:
df['NAME'] = df['NAME'].str.lower()


<ipython-input-35-1a51a7ee85b1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['NAME'] = df['NAME'].str.lower()


In [36]:
df['NAME'] = df['NAME'].apply(lambda x: re.sub(r'[^\w\s]', '', x))


<ipython-input-36-e47d14e35ec3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['NAME'] = df['NAME'].apply(lambda x: re.sub(r'[^\w\s]', '', x))


In [37]:
df['NAME'] = df['NAME'].apply(lambda x: contractions.fix(x))
df['NAME'] = df['NAME'].apply(lambda x: x.strip())


<ipython-input-37-8e73aeaf598c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['NAME'] = df['NAME'].apply(lambda x: contractions.fix(x))
<ipython-input-37-8e73aeaf598c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['NAME'] = df['NAME'].apply(lambda x: x.strip())


In [ ]:
# Mean Pooling - Take attention mask into account for correct averaging
def meanpooling(output, mask):
    embeddings = output[0]  # First element of model_output contains all token embeddings
    mask = mask.unsqueeze(-1).expand(embeddings.size()).float()
    return torch.sum(embeddings * mask, 1) / torch.clamp(mask.sum(1), min=1e-9)

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained("neuml/pubmedbert-base-embeddings")
model = AutoModel.from_pretrained("neuml/pubmedbert-base-embeddings")

def get_sentence_embedding(sentence):

    if pd.isnull(sentence) or sentence == 'missing':
        return np.array([])
    # Tokenize the sentence
    inputs = tokenizer(sentence, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        output = model(**inputs)

    # Perform mean pooling
    embedding = meanpooling(output, inputs['attention_mask'])
    return embedding.squeeze().numpy()  # Convert to NumPy array for easier handling in DataFrame

# Apply the function to each sentence in the 'text_column'
df['NAME_embeddings'] = df['NAME'].apply(get_sentence_embedding)

# for each in df['SMRTDTA_ELEM_VALUE'].unique():
#   print(get_sentence_embedding(each))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/706k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/667 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
print(df[['NAME', 'NAME_embeddings']])


In [ ]:
df.info()


In [ ]:
df


In [ ]:
df.columns


In [ ]:
df = df.drop(columns=['SMRTDTA_tokens'])


In [ ]:
# df_final.to_csv('/content/drive/MyDrive/DATA/Processed_patient_post_op_complications.csv', index=False)